In [1]:
#读取波士顿房价信息：
import numpy as np
import torch
from torch.autograd import Variable

class Boston_price:

    def __init__(self):
        message = []
        with open('boston_housing.txt','r') as f:
            data = f.readlines()

            real_data = data[22:]

        total = []
        for i in real_data:
            a = i.split()
            for j in a:
                total.append(j)

        elements = []
        price = []

        t = 0
        while t < 7084:

            if (t+1) % 14 == 0:
                price.append(total[t])

            else:
                elements.append(total[t])

            t = t + 1

        #对因素进行打包，每13个因素放入一个数组：
        total_ele = []
        while len(elements) > 13:
            temp = elements[:13]
            elements = elements[13:]
            total_ele.append(temp)

        total_ele.append(elements)

        self.total_ele = total_ele
        self.price = price

    def getit_for_dl(self):
        """
        seq代表要划分成的每一个时间段的长度：
        """
        #以上完成波士顿房价的提取处理;接下来把数据全部转换成浮点型：
        total_ele = []
        for i in self.total_ele:
            temp = []
            for j in i:
                temp.append(float(j))

            total_ele.append(temp)

        price = []
        for k in self.price:
            price.append(float(k))


        #先将普通python数组转换为numpy数组:
        ele = np.asarray(total_ele,dtype=float)
        price = np.asarray(price,dtype=float)

        #将numpy数组转换为tensor型：
        ele = torch.from_numpy(ele)
        price = torch.from_numpy(price)

        price = torch.unsqueeze(price,dim=1)

        ele, price = Variable(ele), Variable(price)

        return ele,price

    def getit_for_sk(self):
        """
        seq代表要划分成的每一个时间段的长度：
        """
        #以上完成波士顿房价的提取处理;接下来把数据全部转换成浮点型：
        total_ele = []
        for i in self.total_ele:
            temp = []
            for j in i:
                temp.append(float(j))

            total_ele.append(temp)

        price = []
        for k in self.price:
            price.append(float(k))


        #将普通python数组转换为numpy数组:
        ele = np.asarray(total_ele,dtype=float)
        price = np.asarray(price,dtype=float)


        return ele,price


In [2]:
import torch
import torch.nn as nn

class GRUnet(nn.Module):

    def __init__(self):
        super(GRUnet, self).__init__()

        self.linear1 = nn.Linear(13, 18)

        self.linear2 = nn.Linear(18, 24)

        self.linear3 = nn.Linear(24,32)

        self.linear4 = nn.Linear(32,48)

        self.tan2 = nn.Tanh()

        self.linear5 = nn.Linear(48,32)

        self.tan3 = nn.Tanh()

        self.linear6 = nn.Linear(32,24)

        self.linear7 = nn.Linear(24,18)

        self.linear8 = nn.Linear(18,13)

        self.linear9 = nn.Linear(13, 1)

        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x1 = self.linear1(x)

        x2 = self.linear2(x1)

        x3 = self.linear3(x2)

        x4 = self.linear4(x3)

        x4 = self.tan2(x4)

        x5 = self.linear5(x4)

        x5 = self.tan3(x5)

        x6 = self.linear6(x5)

        x7 = self.linear7(x6)

        x8 = self.linear8(x7)

        x9 = self.linear9(x8)

        return x9

In [3]:
from torch.utils.data import Dataset,DataLoader,TensorDataset
import torch
import matplotlib.pyplot as plt
from sklearn import metrics

bp = Boston_price()
training,label = bp.getit_for_dl()

print(training.shape)
print(label.shape)

deal_dataset = TensorDataset(training,label)
train_loader = DataLoader(dataset=deal_dataset,
                          batch_size = 256,
                          shuffle = True)

net = GRUnet()
net = net.double()
print(net)

loss_func = torch.nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(),lr = 0.0005)

for epoch in range(10001):

    for i, data in enumerate(train_loader):
        x, y = data
        
        prediction = net(x)

        loss = loss_func(prediction, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if epoch % 100 == 0:

        print("完成第",str(epoch),"epoch训练！")

net = net.cpu()
logits = net(training)

l = loss_func(logits,label)

print("MSE:{}".format(l))
print("RMSE:{}".format(torch.sqrt(l)))

y_test = label.numpy()
y_pred = logits.detach().numpy()

mae = metrics.mean_absolute_error(y_test,y_pred)

print("MAE:",mae)

plt.figure(figsize = (15,5))
plt.plot(range(len(y_test)),y_test,'r',label = '测试数据')
plt.plot(range(len(y_pred)),y_pred,'b',label = '预测数据')
plt.legend()
plt.show()

torch.Size([506, 13])
torch.Size([506, 1])
GRUnet(
  (linear1): Linear(in_features=13, out_features=18, bias=True)
  (linear2): Linear(in_features=18, out_features=24, bias=True)
  (linear3): Linear(in_features=24, out_features=32, bias=True)
  (linear4): Linear(in_features=32, out_features=48, bias=True)
  (tan2): Tanh()
  (linear5): Linear(in_features=48, out_features=32, bias=True)
  (tan3): Tanh()
  (linear6): Linear(in_features=32, out_features=24, bias=True)
  (linear7): Linear(in_features=24, out_features=18, bias=True)
  (linear8): Linear(in_features=18, out_features=13, bias=True)
  (linear9): Linear(in_features=13, out_features=1, bias=True)
  (relu): ReLU(inplace=True)
)
完成第 0 epoch训练！
完成第 100 epoch训练！
完成第 200 epoch训练！
完成第 300 epoch训练！
完成第 400 epoch训练！
完成第 500 epoch训练！
完成第 600 epoch训练！
完成第 700 epoch训练！
完成第 800 epoch训练！
完成第 900 epoch训练！
完成第 1000 epoch训练！
完成第 1100 epoch训练！
完成第 1200 epoch训练！
完成第 1300 epoch训练！
完成第 1400 epoch训练！
完成第 1500 epoch训练！
完成第 1600 epoch训练！


KeyboardInterrupt: 